### Configuration

In [2]:
req_pkgs = c('dplyr', 'ggplot2', 'repr')
lapply(req_pkgs, require, character.only = TRUE)
options(repr.plot.width=10, repr.plot.height=8)
options(scipen = 999)

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

## Create dataset where we know the exact coefficient of a variable

In [3]:
train_df = data.frame(y_var = rnorm(1000, 100, 10)) %>%
mutate(categ_var = c(rep("A", 500), rep("B", 200), rep("C", 300))) %>%
mutate(offset_categ_var = ifelse(categ_var == "A", 10,
                                ifelse(categ_var == "B", -10, 0))) %>%
mutate(y_var = y_var + offset_categ_var)

train_df %>% head()

y_var,categ_var,offset_categ_var
<dbl>,<chr>,<dbl>
111.12310,A,10
89.49637,A,10
101.60326,A,10
105.90712,A,10
104.52851,A,10
114.62386,A,10


In [4]:
ggplot(train_df, aes(x = factor(categ_var), y = y_var, color = factor(categ_var), fill = factor(categ_var))) +
theme_bw() +
geom_violin(alpha = 0.3) +
theme(legend.position = "bottom")

## Fit on Intercept Only

In [5]:
fit1 = glm(y_var ~ 1, data = train_df)
summary(fit1)


Call:
glm(formula = y_var ~ 1, data = train_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-40.485   -7.853    0.153    8.243   40.387  

Coefficients:
            Estimate Std. Error t value            Pr(>|t|)    
(Intercept) 103.1295     0.3882   265.6 <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 150.7347)

    Null deviance: 150584  on 999  degrees of freedom
Residual deviance: 150584  on 999  degrees of freedom
AIC: 7856.4

Number of Fisher Scoring iterations: 2


## Fit on Intercept with Offset

In [6]:
fit2 = glm(y_var ~ offset(offset_categ_var), data = train_df)
summary(fit2)


Call:
glm(formula = y_var ~ offset(offset_categ_var), data = train_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-27.766   -6.934    0.237    6.633   33.387  

Coefficients:
            Estimate Std. Error t value            Pr(>|t|)    
(Intercept) 100.1295     0.3082   324.8 <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 95.00859)

    Null deviance: 94914  on 999  degrees of freedom
Residual deviance: 94914  on 999  degrees of freedom
AIC: 7394.8

Number of Fisher Scoring iterations: 2


##### notice that AIC decreased, suggesting an improved fit

## Let's compare residual distributions

In [7]:
resid_df = data.frame(model = c(rep("Intercept Only", 1000), rep("Intercept with Offset", 1000)),
                      residual = c(fit1$residual, fit2$residual),
                      categ_var = c(train_df$categ_var, train_df$categ_var))

In [8]:
ggplot(resid_df, aes(residual, color = categ_var, fill =categ_var)) +
theme_bw() +
geom_density(alpha = 0.3) +
facet_wrap(~model) +
theme(legend.position = 'bottom')

#### notice that the offset model does not have bias in the residuals